In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Define the paths to the folders containing images
benign_path = r'Benign'  # Replace with your actual path
malignant_path = r'Malignant'  # Replace with your actual path

# Image dimensions (resized for CNN)
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels
def load_images_from_folders(benign_path, malignant_path):
    images = []
    labels = []

    # Load Benign images
    for filename in os.listdir(benign_path):
        if filename.startswith('.'):  # Skip hidden files/folders
            continue
        file_path = os.path.join(benign_path, filename)
        try:
            img = load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(0)  # Label for Benign
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")

    # Load Malignant images
    for filename in os.listdir(malignant_path):
        if filename.startswith('.'):  # Skip hidden files/folders
            continue
        file_path = os.path.join(malignant_path, filename)
        try:
            img = load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(1)  # Label for Malignant
        except Exception as e:
            print(f"Error loading image {file_path}: {e}")

    return np.array(images), np.array(labels)

# Load the images and labels
X_images, y_labels = load_images_from_folders(benign_path, malignant_path)

# Normalize the pixel values to [0, 1]
X_images = X_images / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Output layer for 2 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation')


Epoch 1/30


ValueError: in user code:

    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\dmill\anaconda3\envs\AI_PROJECT\lib\site-packages\keras\engine\input_spec.p